# Relax Data Challenge

### Dependencies & data

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

sns.set()
%matplotlib inline

In [3]:
# Usage data
usage_df = pd.read_csv('takehome_user_engagement.csv', parse_dates=['time_stamp'])
print(usage_df.info())
usage_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time_stamp  207917 non-null  datetime64[ns]
 1   user_id     207917 non-null  int64         
 2   visited     207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB
None


,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1


In [4]:
# User data
user_df = pd.read_csv('takehome_users.csv', parse_dates=['creation_time'], encoding='latin-1')
print(user_df.info())
user_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  8823 non-null   float64       
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          6417 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 937.6+ KB
None


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [5]:
# Rename `object_id` to `user_id`
user_df.rename(columns={'object_id': 'user_id'}, inplace=True)

# Convert `last_session_creation_time` to datetime
user_df['last_session_creation_time'] = pd.to_datetime(user_df.last_session_creation_time, unit='s')
user_df.head(2)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0


### Label adopted users and count usage for each user

In [6]:
# Group visits by each user's weekly visits
weekly_usage_df = usage_df.groupby(['user_id', pd.Grouper(key='time_stamp', freq='7d')]).count()

# Filter for users who visited at least 3 times during any week
adopted_usage_df = weekly_usage_df[weekly_usage_df['visited'] >= 3]
adopted_users = adopted_usage_df.index.get_level_values(0).unique().values

print(len(adopted_users))
weekly_usage_df.head(2)

1445


,,visited
user_id,time_stamp,
1,2014-04-17,1
2,2013-11-14,1


In [7]:
# Create label for adopted users
# user_df['adopted_user'] = np.where(user_df['user_id'].isin(adopted_users), 1, 0)
user_df['adopted_user'] = user_df['user_id'].isin(adopted_users).astype(int)
assert user_df['adopted_user'].sum() == len(adopted_users) # confirm equal length
user_df.head(2)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,0


In [8]:
# Create a feature for the number of times each user logged in
usage_count_df = usage_df.groupby('user_id', as_index=False).sum()
user_df = user_df.merge(usage_count_df, on='user_id', how='left')
user_df['visited'] = user_df['visited'].fillna(0).astype(int) # fill missing with 0
user_df.head(2)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,0,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,0,14


### Split data

In [9]:
# Split features and label
X = user_df.drop('adopted_user', axis=1)
y = user_df['adopted_user'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9000, 11), (3000, 11), (9000,), (3000,))

In [10]:
# Class (im)balance
y_train.value_counts()

0    7916
1    1084
Name: adopted_user, dtype: int64

### Resample data

In [11]:
def join_label_into_feats(X, y):
    df = X.merge(y, left_index=True, right_index=True)
    return df

# Test function
train = join_label_into_feats(X_train, y_train)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9000 entries, 906 to 920
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     9000 non-null   int64         
 1   creation_time               9000 non-null   datetime64[ns]
 2   name                        9000 non-null   object        
 3   email                       9000 non-null   object        
 4   creation_source             9000 non-null   object        
 5   last_session_creation_time  6603 non-null   datetime64[ns]
 6   opted_in_to_mailing_list    9000 non-null   int64         
 7   enabled_for_marketing_drip  9000 non-null   int64         
 8   org_id                      9000 non-null   int64         
 9   invited_by_user_id          4843 non-null   float64       
 10  visited                     9000 non-null   int64         
 11  adopted_user                9000 non-null   int64      

In [12]:
# Split train data by label
train0 = train[train['adopted_user'] == 0].copy()
train1 = train[train['adopted_user'] == 1].copy()
train.shape, train0.shape, train1.shape

((9000, 12), (7916, 12), (1084, 12))

In [13]:
# Upsample positive labels
train1_resampled = train1.sample(train0.shape[0], replace=True, random_state=10)

# Combine training data
train_resampled = pd.concat([train0, train1_resampled]).reset_index(drop=True)
train_resampled['adopted_user'].value_counts()

0    7916
1    7916
Name: adopted_user, dtype: int64

In [14]:
# Split features and label
X_train_resampled = train_resampled.drop('adopted_user', axis=1)
y_train_resampled = train_resampled['adopted_user'].copy()
X_train_resampled.shape, y_train_resampled.shape

((15832, 11), (15832,))

In [15]:
# Copy train set for preprocessing
X_train_preprocessed = X_train_resampled.copy()
y_train_preprocessed = y_train_resampled.copy()
X_train_preprocessed.shape, y_train_preprocessed.shape

((15832, 11), (15832,))

### Datetime EDA

In [1]:
# Create a feature for each user's first day of use
first_use = dict(usage_df.groupby('user_id')['time_stamp'].min()) # save for feature pipeline
X_train_resampled['first_use'] = X_train_resampled['user_id'].map(first_use)

# Create a feature for the number of days between account creation and first use
X_train_resampled['days_to_first_use'] = (X_train_resampled['first_use'] - X_train_resampled['creation_time']).dt.days
X_train_resampled['days_to_first_use'].fillna(999, inplace=True)
X_train_resampled['days_to_first_use'].value_counts()

NameError: name 'usage_df' is not defined

In [ ]:
# Create a feature for same day use
X_train_resampled['same_day_use'] = (X_train_resampled['days_to_first_use'] == 0).astype(int)
X_train_resampled['same_day_use'].value_counts()

In [ ]:
# Join label into features
train_resampled = join_label_into_feats(X_train_resampled, y_train_resampled)

# User adoption rate of users who login within the first n days
ar_1d = train_resampled[train_resampled['days_to_first_use'] < 1]['adopted_user'].mean()
ar_7d = train_resampled[train_resampled['days_to_first_use'] < 7]['adopted_user'].mean()
ar_30d = train_resampled[train_resampled['days_to_first_use'] < 30]['adopted_user'].mean()

print(f'{ar_1d * 100:.2f}% of users who login within the same DAY as creating an account are adopted users.')
print(f'{ar_7d * 100:.2f}% of users who login within the same WEEK as creating an account are adopted users.')
print(f'{ar_30d * 100:.2f}% of users who login within the same MONTH as creating an account are adopted users.')

In [ ]:
# Check adoption rate of each signup month
monthly_adoption_df = train_resampled.groupby(pd.Grouper(key='creation_time', freq='M'))[['adopted_user']].mean().reset_index()
monthly_adoption_df['creation_time'] = monthly_adoption_df['creation_time'].dt.strftime('%Y-%m')
monthly_adoption_df.columns = ['creation_month', 'adoption_rate']

# Plot monthly adoption
plt.figure(figsize=(16, 4))
sns.lineplot(data=monthly_adoption_df, x='creation_month', y='adoption_rate')
plt.title('Monthly Adoption Rate', fontsize=16)
plt.xticks(ticks=range(monthly_adoption_df.shape[0]), labels=monthly_adoption_df['creation_month'], rotation=45, ha='right')
plt.xlim((0, monthly_adoption_df.shape[0] - 1))

monthly_adoption_df

### Categorical EDA

In [ ]:
# Simplify `invited_by_user_id` to a boolean feature that indicates whether the user was invited
X_train_resampled['invited'] = (X_train_resampled['invited_by_user_id'].notnull()).astype(int)

# Extract email domain
X_train_resampled['email_domain'] = X_train_resampled['email'].str.split('@', expand=True)[1]
email_domains = X_train_resampled['email_domain'].value_counts()
email_domains[email_domains > 10]

In [ ]:
# Group rare domains into a single category
common_domains = email_domains[email_domains > 20].index.values # save for feature pipeline
X_train_resampled['email_domain'] = X_train_resampled['email_domain'].apply(lambda d: d if d in common_domains else 'other')
X_train_resampled['email_domain'].value_counts()

In [ ]:
# Join label into features
train_resampled = join_label_into_feats(X_train_resampled, y_train_resampled)

# User adoption and average logins by email domain
fig1, ax1 = plt.subplots(1, 2, figsize=(16, 4))
sns.countplot(data=train_resampled, x='email_domain', hue='adopted_user', ax=ax1[0])
sns.barplot(data=train_resampled, x='email_domain', y='visited', ax=ax1[1])
ax1[0].set_title('User Adoption by Email Domain', fontsize=16)
ax1[1].set_title('Average Logins by Email Domain', fontsize=16)
ax1[0].set_xticklabels(ax1[0].get_xticklabels(), rotation=30, ha='right')
ax1[1].set_xticklabels(ax1[1].get_xticklabels(), rotation=30, ha='right')

# User adoption rate and average logins by email domain
train_resampled.groupby('email_domain')[['adopted_user', 'visited']].mean()

In [ ]:
# User adoption and average logins by creation source
fig2, ax2 = plt.subplots(1, 2, figsize=(16, 4))
sns.countplot(data=train_resampled, x='creation_source', hue='adopted_user', ax=ax2[0])
sns.barplot(data=train_resampled, x='creation_source', y='visited', ax=ax2[1])
ax2[0].set_title('User Adoption by Creation Source', fontsize=16)
ax2[1].set_title('Average Logins by Creation Source', fontsize=16)
ax2[0].set_xticklabels(ax2[0].get_xticklabels(), rotation=30, ha='right')
ax2[1].set_xticklabels(ax2[1].get_xticklabels(), rotation=30, ha='right')

# User adoption rate and average logins by creation source
user_df.groupby('creation_source')[['adopted_user', 'visited']].mean()

In [ ]:
# Group organizations by size
org_users = X_train_resampled['org_id'].value_counts()
pd.qcut(org_users, 10)

In [ ]:
# Group organizations into 9 bins depending on size
bin_size = 9
org_cats = pd.qcut(org_users, bin_size, labels=range(bin_size)).to_dict() # save for feature pipeline
X_train_resampled['org_size'] = X_train_resampled['org_id'].map(org_cats)

# Group categories 0 and 1
X_train_resampled.loc[X_train_resampled['org_size'] == 0, 'org_size'] = 1
X_train_resampled['org_size'].value_counts()

In [ ]:
# Join label into features
train_resampled = join_label_into_feats(X_train_resampled, y_train_resampled)

# Adoption rate by organization
org_adoption_df = train_resampled.groupby('org_size')[['adopted_user', 'visited']].mean().sort_values('adopted_user', ascending=False)
org_adoption_df['adopted_user'].plot(kind='barh', figsize=(8, 8))
plt.title('Adoption Rate of Different Organization Size Classes', fontsize=16)
plt.xlabel('adoption_rate')

org_adoption_df

### Encode categorical variables

In [ ]:
# Create an organization-to-adoption-rate mapping
org_ar = org_adoption_df['adopted_user'].sort_index().to_dict() # save for feature pipeline
print(org_ar)

# Encode `org_id` with their adoption rate
X_train_resampled['org_enc'] = X_train_resampled['org_size'].map(org_ar)
X_train_resampled.head(2)

In [ ]:
# Encode creation_source
src_ar = train_resampled.groupby('creation_source')['adopted_user'].mean().to_dict() # save for feature pipeline
print(src_ar)

# Encode `creation_source` with their adoption rate
X_train_resampled['creation_enc'] = X_train_resampled['creation_source'].map(src_ar)
X_train_resampled.head(2)

In [ ]:
# Encode email_domain
dom_ar = train_resampled.groupby('email_domain')['adopted_user'].mean().to_dict() # save for feature pipeline
X_train_resampled['email_enc'] = X_train_resampled['email_domain'].map(dom_ar)
X_train_resampled.head(2)

### Select features

In [ ]:
# Select features for train set
selected_cols = ['creation_enc', 'email_enc', 'org_enc', 'invited', 'same_day_use', 
                 'opted_in_to_mailing_list', 'enabled_for_marketing_drip']
X_train_preprocessed = X_train_resampled[selected_cols].copy()
y_train_preprocessed = y_train_resampled.copy()

X_train_preprocessed.shape, y_train_preprocessed.shape

### Feature pipeline

In [ ]:
def feature_pipeline(data, first_use_mapping, domain_list, domain_encoding, 
                     org_bins, org_encoding, source_encoding, selected_features):
    
    """
    Feature pipeline:
    - same_day_use
        > first_use (encode user_id w/ `first_use`) 
        > days_to_first_use (diff first_use and creation_time in days, fill na w/ 999) 
        > same_day_use (binary encode days_to_first_use)
    - invited (binary encode invited_by_user_id)
    - email_enc
        > email_domain (extract domain from email, rare-category encode email_domain w/ `common_domains`) 
        > email_enc (mean-target encode email_domain w/ `dom_ar`)
    - org_enc
        > org_size (bin org_id value counts w/ `org_cats`, combine bins 0 and 1)
        > org_enc (mean-target encode org_size w/ `org_ar`)
    - creation_enc (mean-target encode creation_source w/ `src_ar`)
    - select features: 'creation_enc', 'email_enc', 'org_enc', 'invited', 'same_day_use', 
                       'opted_in_to_mailing_list', 'enabled_for_marketing_drip'
    """
    
    data = data.copy()
    
    # Create binary feature for same day use
    data['first_use'] = data['user_id'].map(first_use_mapping) # first day of use
    data['days_to_first_use'] = (data['first_use'] - data['creation_time']).dt.days # number of days to first use
    data['days_to_first_use'].fillna(999, inplace=True)
    data['same_day_use'] = (data['days_to_first_use'] == 0).astype(int)
    
    # Create a binary feature for invited
    data['invited'] = (data['invited_by_user_id'].notnull()).astype(int)
    
    # Encode email domains with their adoption rates
    data['email_domain'] = data['email'].str.split('@', expand=True)[1] # email domain
    data['email_domain'] = data['email_domain'].apply(lambda d: d if d in domain_list else 'other') # encode rare categories
    data['email_enc'] = data['email_domain'].map(domain_encoding)
    
    # Encode organization sizes with their adoption rates
    data['org_size'] = data['org_id'].map(org_bins) # group organizations into 9 bins
    data.loc[data['org_size'] == 0, 'org_size'] = 1 # group bins 0 and 1
    data['org_enc'] = data['org_size'].map(org_encoding)

    # Encode creation sources with their adoption rates
    data['creation_enc'] = data['creation_source'].map(source_encoding)
    
    # Select features
    data = data[selected_features].copy()
    
    return data


# Test pipeline
X_test_preprocessed = feature_pipeline(X_test, first_use, common_domains, dom_ar, org_cats, org_ar, src_ar, selected_cols)
y_test_preprocessed = y_test.copy()
X_test_preprocessed.head(2)

### Modeling 1

In [ ]:
# Logistic regression
lr = LogisticRegression(max_iter=1e4, random_state=0)
lr.fit(X_train, y_train)
lr_pred_train = lr.predict(X_train)
lr_pred_test = lr.predict(X_test)

print('Train')
print('Accuracy:', lr.score(X_train, y_train))
print('ROC AUC:', roc_auc_score(y_train, lr_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train, lr_pred_train))

print('\nTest')
print('Accuracy:', lr.score(X_test, y_test))
print('ROC AUC:', roc_auc_score(y_test, lr_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, lr_pred_test))

In [ ]:
# Decision tree grid search
dt = DecisionTreeClassifier(random_state=0)
params = {'max_depth': range(2, 22, 2), 'min_samples_split': range(2, 22, 2), 'min_samples_leaf': range(2, 22, 2)}
gs = GridSearchCV(dt, params, scoring='roc_auc', cv=5, n_jobs=10)
gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)
gs.best_estimator_

In [ ]:
# Decision tree
dt = gs.best_estimator_
dt.fit(X_train, y_train)
dt_pred_train = dt.predict(X_train)
dt_pred_test = dt.predict(X_test)

print('Train')
print('Score:', dt.score(X_train, y_train))
print('ROC AUC:', roc_auc_score(y_train, dt_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train, dt_pred_train))

print('\nTest')
print('Score:', dt.score(X_test, y_test))
print('ROC AUC:', roc_auc_score(y_test, dt_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, dt_pred_test))

In [ ]:
# Feature coefficients
pd.DataFrame(np.array([X_train.columns.values, lr.coef_[0], dt.feature_importances_]).T, 
             columns=['feature', 'lr_coef', 'dt_coef'])

### Modeling 2 (with resampled data)

In [ ]:
# Split data by label
train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
train0 = train[train['adopted_user'] == 0]
train1 = train[train['adopted_user'] == 1]
train.shape, train0.shape, train1.shape

In [ ]:
# Upsample positive labels
train1_resampled = train1.sample(train0.shape[0], replace=True)

# Combine training data
train_resampled = pd.concat([train0, train1_resampled])
train_resampled['adopted_user'].value_counts()

In [ ]:
# Split features and label
X_train_resampled = train_resampled.drop('adopted_user', axis=1).copy()
y_train_resampled = train_resampled['adopted_user'].copy()
X_train_resampled.shape, y_train_resampled.shape

In [ ]:
# Logistic regression
lr_resampled = LogisticRegression(max_iter=1e4, random_state=0)
lr_resampled.fit(X_train_resampled, y_train_resampled)
lr_resampled_pred_train = lr_resampled.predict(X_train_resampled)
lr_resampled_pred_test = lr_resampled.predict(X_test)

print('Train')
print('Accuracy:', lr_resampled.score(X_train_resampled, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, lr_resampled_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, lr_resampled_pred_train))

print('\nTest')
print('Accuracy:', lr_resampled.score(X_test, y_test))
print('ROC AUC:', roc_auc_score(y_test, lr_resampled_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, lr_resampled_pred_test))

In [ ]:
# Decision tree grid search
dt_resampled = DecisionTreeClassifier(random_state=0)
params_resampled = {'max_depth': range(2, 22, 2), 'min_samples_split': range(2, 22, 2), 'min_samples_leaf': range(2, 22, 2)}
gs_resampled = GridSearchCV(dt_resampled, params_resampled, scoring='roc_auc', cv=5, n_jobs=10)
gs_resampled.fit(X_train_resampled, y_train_resampled)

print(gs_resampled.best_score_)
print(gs_resampled.best_params_)
gs_resampled.best_estimator_

In [ ]:
# Decision tree
dt_resampled = gs_resampled.best_estimator_
dt_resampled.fit(X_train_resampled, y_train_resampled)
dt_resampled_pred_train = dt_resampled.predict(X_train_resampled)
dt_resampled_pred_test = dt_resampled.predict(X_test)

print('Train')
print('Score:', dt_resampled.score(X_train_resampled, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, dt_resampled_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, dt_resampled_pred_train))

print('\nTest')
print('Score:', dt_resampled.score(X_test, y_test))
print('ROC AUC:', roc_auc_score(y_test, dt_resampled_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, dt_resampled_pred_test))

In [ ]:
# Feature coefficients
pd.DataFrame(np.array([X_train_resampled.columns.values, lr_resampled.coef_[0], dt_resampled.feature_importances_]).T, 
             columns=['feature', 'lr_coef', 'dt_coef'])

### Modeling 3 (with a binary feature subset)

In [ ]:
# Filter for binary features
X_train_bin = X_train_resampled[['invited', 'mailing_list', 'marketing_drip']].copy()
X_test_bin = X_test[['invited', 'mailing_list', 'marketing_drip']].copy()
X_train_bin.shape, X_test_bin.shape

In [ ]:
# Logistic regression
lr_bin = LogisticRegression(max_iter=1e4, random_state=0)
lr_bin.fit(X_train_bin, y_train_resampled)
lr_bin_pred_train = lr_bin.predict(X_train_bin)
lr_bin_pred_test = lr_bin.predict(X_test_bin)

print('Train')
print('Accuracy:', lr_bin.score(X_train_bin, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, lr_bin_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, lr_bin_pred_train))

print('\nTest')
print('Accuracy:', lr_bin.score(X_test_bin, y_test))
print('ROC AUC:', roc_auc_score(y_test, lr_bin_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, lr_bin_pred_test))

In [ ]:
# Decision tree grid search
dt_bin = DecisionTreeClassifier(random_state=0)
params_bin = {'max_depth': range(2, 22, 2), 'min_samples_split': range(2, 22, 2), 'min_samples_leaf': range(2, 22, 2)}
gs_bin = GridSearchCV(dt_bin, params_bin, scoring='roc_auc', cv=5, n_jobs=10)
gs_bin.fit(X_train_bin, y_train_resampled)

print(gs_bin.best_score_)
print(gs_bin.best_params_)
gs_bin.best_estimator_

In [ ]:
# Decision tree
dt_bin = gs_bin.best_estimator_
dt_bin.fit(X_train_bin, y_train_resampled)
dt_bin_pred_train = dt_bin.predict(X_train_bin)
dt_bin_pred_test = dt_bin.predict(X_test_bin)

print('Train')
print('Score:', dt_bin.score(X_train_bin, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, dt_bin_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, dt_bin_pred_train))

print('\nTest')
print('Score:', dt_bin.score(X_test_bin, y_test))
print('ROC AUC:', roc_auc_score(y_test, dt_bin_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, dt_bin_pred_test))

In [ ]:
# Feature coefficients
pd.DataFrame(np.array([X_train_bin.columns.values, lr_bin.coef_[0], dt_bin.feature_importances_]).T, 
             columns=['feature', 'lr_coef', 'dt_coef'])

### Modeling 4 (with encoded feature subset)

In [ ]:
# Filter for encoded features
X_train_enc = X_train_resampled[['creation_encoding', 'domain_encoding', 'org_encoding']].copy()
X_test_enc = X_test[['creation_encoding', 'domain_encoding', 'org_encoding']].copy()
X_train_enc.shape, X_test_enc.shape

In [ ]:
# Logistic regression
lr_enc = LogisticRegression(max_iter=1e4, random_state=0)
lr_enc.fit(X_train_enc, y_train_resampled)
lr_enc_pred_train = lr_enc.predict(X_train_enc)
lr_enc_pred_test = lr_enc.predict(X_test_enc)

print('Train')
print('Accuracy:', lr_enc.score(X_train_enc, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, lr_enc_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, lr_enc_pred_train))

print('\nTest')
print('Accuracy:', lr_enc.score(X_test_enc, y_test))
print('ROC AUC:', roc_auc_score(y_test, lr_enc_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, lr_enc_pred_test))

In [ ]:
# Decision tree grid search
dt_enc = DecisionTreeClassifier(random_state=0)
params_enc = {'max_depth': range(2, 22, 2), 'min_samples_split': range(2, 22, 2), 'min_samples_leaf': range(2, 22, 2)}
gs_enc = GridSearchCV(dt_enc, params_enc, scoring='roc_auc', cv=5, n_jobs=10)
gs_enc.fit(X_train_enc, y_train_resampled)

print(gs_enc.best_score_)
print(gs_enc.best_params_)
gs_enc.best_estimator_

In [ ]:
# Decision tree
dt_enc = gs_enc.best_estimator_
dt_enc.fit(X_train_enc, y_train_resampled)
dt_enc_pred_train = dt_enc.predict(X_train_enc)
dt_enc_pred_test = dt_enc.predict(X_test_enc)

print('Train')
print('Score:', dt_enc.score(X_train_enc, y_train_resampled))
print('ROC AUC:', roc_auc_score(y_train_resampled, dt_enc_pred_train))
print('Confusion matrix:\n', confusion_matrix(y_train_resampled, dt_enc_pred_train))

print('\nTest')
print('Score:', dt_enc.score(X_test_enc, y_test))
print('ROC AUC:', roc_auc_score(y_test, dt_enc_pred_test))
print('Confusion matrix:\n', confusion_matrix(y_test, dt_enc_pred_test))

In [ ]:
# Feature coefficients
pd.DataFrame(np.array([X_train_enc.columns.values, lr_enc.coef_[0], dt_enc.feature_importances_]).T, 
             columns=['feature', 'lr_coef', 'dt_coef'])